# Graph Magma Class 

In [9]:
class graphMagma:
    from sage.matrix.constructor import block_matrix, zero_matrix, identity_matrix, matrix
    from sage.rings.integer_ring import ZZ
    #Create a global variable to store the generators
    generators = []
    Inc = matrix(ZZ, 0, 0, [])
    multiplication_table = matrix(SR,0,0,[])
    #Create a global variable to store the graph
    graph = []
    def __init__(self, graph:list):
        self.graph = graph
        self.generators = self.getGenerators()
        self.Inc = self.getIncidenceMatrix()
        self.multiplication_table = self.getMultiplicationTable()
    def getGenerators(self):
        self.generators = []
        for i in range(len(self.graph)):
            for j in range(self.graph[i][1]):
                self.generators.append(var(f"a_{i}_{j}", latex_name=fr"\alpha_{{{i},{j}}}"))
        return self.generators
    def getGraph(self):
        return self.graph
    def incidenceMatrix(self):
        G=self.graph
        I=self.Inc
        for g in G:
            if g[0] == 'K':
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1], g[1], [1]*(g[1]*g[1]))]])
            else:
                I = block_matrix([[I, matrix(I.nrows(), g[1],[1]*(I.nrows()*g[1]))], [zero_matrix(g[1], I.ncols()), matrix(g[1],g[1])]])  
        self.Inc = I
    def getIncidenceMatrix(self):
        self.incidenceMatrix()
        return self.Inc  
    def multiplicationTable(self):
        I=self.Inc
        M=matrix(SR,I.nrows(),I.nrows())
        self.getGenerators()
        for i in range(I.nrows()):
            for j in range(I.nrows()):
                if I[i,j]==0:
                    M[i,j]=self.generators[j]
                else:
                    M[i,j]=self.generators[i]
        self.multiplication_table = M
    def getMultiplicationTable(self):
        self.multiplicationTable()
        return self.multiplication_table

# Graph Magma Algebra Class

In [ ]:
from sage.algebras.free_algebra import FreeAlgebra
from sage.algebras.free_algebra_quotient import FreeAlgebraQuotient
from sage.structure.element import Element
from random import choice
relations=[]
class GraphMagmaAlg:
    def __init__(self,base_ring,GM:graphMagma):
        R=FreeAlgebraQuotient
        self.base_ring=base_ring
        self.generators=GM.getGenerators()
        self.generators_str=[str(g) for g in self.generators]
        self.graph=GM.getGraph()
        self.multiplication_table=GM.getMultiplicationTable()
        # Determine the number of generators from the list
        n = len(self.generators)
        # Create a string for the generator names
        generator_names_str =', '.join(self.generators_str)
        #g_str=f"R.<{generator_names_str}>=FreeAlgebraQuotient(A,self.mons,self.mats)";print(g_str)
        # Create the FreeAlgebra instance
        A = FreeAlgebra(base_ring, n,generator_names_str)
        self.alg=A
        F=A.monoid() #generator_names_str=F.gens();print(F.gens())
        self.mons=[F(1)]+list(F.gens())
        self.mats=self.build_relations()
        self.magma_algebra=FreeAlgebraQuotient(A, self.mons, self.mats, names=self.generators_str)
        splt=[g[1] for g in self.graph]
        L=list(self.magma_algebra.gens()).copy()
        self.gens=self.split_list(L,list(splt))
        self.local_decomposition=self.getDecomposition()
        
        
    
    def __contains__(self, item):
        return item in self.magma_algebra
    
    def __str__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"
    def __repr__(self):
        return f"GraphMagmaAlg with generators {list(self.generators_str)}"

    def build_relations(self):
        act=[];L=[];idx=0
        n=len(self.generators)
        self.relations=[];M=MatrixSpace(self.base_ring,n+1);I=identity_matrix(n+1)
        T=self.multiplication_table
        for j in range(T.nrows()):
            L=list(I[j+1])
            for i in range(T.ncols()):
                idx=self.generators.index(T[i,j])
                L+=list(I[idx+1])
            act.append(M(L))
        return act
            
    def split_list(self,L, lengths):
        result = []
        index = 0
        for length in lengths:
            sublist = L[index:index + length]
            result.append(sublist)
            index += length
        return result

    def getDecomposition(self):
        R=self.magma_algebra;g=self.gens
        L=[R(f[0]) for f in g]
        ideal_generators=[L[0]]+[L[i]-L[i-1] for i in [1..len(L)-1]]+[R(1)-L[-1]]
        return ideal_generators

    def getSimpleLeftIdeals(self,n:int):
        '''This function gives the generators of the simple left ideals in a particular local projective component.
        n represents the index of the local projective component.     '''
        R=self.magma_algebra;G=self.graph;L=self.gens
        print('the generators of the simple left ideals in the local projective component #'+str(n+1)+' are:')
        if n==0:
            if G[0][0]=='N':
                return [L[0][0]]
            else:
                return [L[0][i]-L[0][0] for i in range(1,len(L[0]))]
        else:
            if n<len(G)-1:
                sub_graph=G[n:n+2]
                if sub_graph[0][0]=='K' and sub_graph[1][0]=='K':
                    if sub_graph[1][1]==1:
                        return [L[n+1][0]-L[n][0]]
                    else:
                        return [L[n+1][0]-L[n+1][i] for i in range(1,len(L[n+1]))]
                elif sub_graph[0][0]=='K' and sub_graph[1][0]=='N':
                    return [L[n+1][0]-L[n][0]]
                elif sub_graph[0][0]=="N" and sub_graph[1][0]=="K":
                    if sub_graph[0][1]==sub_graph[1][1]:
                        return [L[n+1][0]-L[n][0]]
                    elif sub_graph[0][1]>1 and sub_graph[1][1]==1:
                        return [L[n][i]-L[n][0] for i in range(1,len(L[n]))]
                    elif sub_graph[0][1]==1 and sub_graph[1][1]>1:
                        return [L[n+1][0]-L[n+1][i] for i in range(1,len(L[n+1]))] 
                    else:
                        return [L[n][i]-L[n][0] for i in range(1,len(L[n]))]+[L[n+1][0]-L[n+1][i] for i in range(1,len(L[n+1]))]
                elif sub_graph[0][0]=="N" and sub_graph[1][0]=="N":
                    if sub_graph[0][1]==1:
                        return [L[n+1][0]-L[n][0]]
                    else:
                        return [L[n][i]-L[n][0] for i in range(1,len(L[n]))]
            else:
                if G[n-1][0]=='N':
                    return [L[n-1][i]-L[n-1][0] for i in range(1,len(L[n-1]))]
                else:
                    return [1-L[n-1][0]]
        




In [118]:
M=graphMagma([('K',2),('N',3),('K',4)])
GM=GraphMagmaAlg(QQ,M);GM.local_decomposition

[a_0_0, -a_0_0 + a_1_0, -a_1_0 + a_2_0, 1 - a_2_0]

In [119]:
GM.getSimpleLeftIdeals(0), GM.getSimpleLeftIdeals(1),  GM.getSimpleLeftIdeals(2), GM.getSimpleLeftIdeals(3)

the generators of the simple left ideals in the local projective component #1 are:
the generators of the simple left ideals in the local projective component #2 are:
the generators of the simple left ideals in the local projective component #3 are:
the generators of the simple left ideals in the local projective component #4 are:


([-a_0_0 + a_0_1],
 [-a_1_0 + a_1_1, -a_1_0 + a_1_2, a_2_0 - a_2_1, a_2_0 - a_2_2, a_2_0 - a_2_3],
 [-a_1_0 + a_1_1, -a_1_0 + a_1_2],
 [1 - a_2_0])

In [48]:
R=GM.magma_algebra
R

Free algebra quotient on 9 generators ('a_0_0', 'a_0_1', 'a_1_0', 'a_1_1', 'a_1_2', 'a_2_0', 'a_2_1', 'a_2_2', 'a_2_3') and dimension 10 over Rational Field

In [65]:
g=GM.gens;g

[[a_0_0, a_0_1], [a_1_0, a_1_1, a_1_2], [a_2_0, a_2_1, a_2_2, a_2_3]]

In [54]:
g[2][0]*g[1][1]

a_1_1

In [11]:
g

[[a_0_0, a_0_1], [a_1_0, a_1_1, a_1_2]]

In [13]:
(1-g[0][0])**2==1-g[0][0]

True